# 5. Getting Verinym

It is important to understand that earlier created **Faber** DID is not, in and of itself, the same thing as self-sovereign identity. This DID must be used only for secure interaction with the **Steward**.
After the connection is established **Faber** must create new DID record that he will use as Verinym in the Ledger.

1. **Faber** creates a new DID in its wallet by calling ``did.create_and_store_my_did``.
    ```python        
    # Faber Agent
    (faber_did, faber_key) = await did.create_and_store_my_did(faber_wallet, "{}")
    ```

2. **Faber** prepares the message that will contain the created DID and verkey.
    ```python        
    # Faber Agent
    faber_did_info_json = json.dumps({
        'did': faber_did,
        'verkey': faber_key
    })
    ```

3. **Faber** authenticates and encrypts the message by calling ``crypto.auth_crypt`` using verkeys created for secure communication with **Steward**. The Authenticated-encryption schema is designed for the sending of a confidential message specifically for a Recipient, using the Sender's public key. Using the Recipient's public key, the Sender can compute a shared secret key. Using the Sender's public key and his secret key, the Recipient can compute the exact same shared secret key. That shared secret key can be used to verify that the encrypted message was not tampered with, before eventually decrypting it.
    ```python        
    # Faber Agent
    authcrypted_faber_did_info_json = \
        await crypto.auth_crypt(faber_wallet, faber_steward_key, steward_faber_key, faber_did_info_json.encode('utf-8'))
    ```

4. **Faber** sends the encrypted message to the **Steward**.

5. **Steward** decrypts the received message by calling ``crypto.auth_decrypt``.
    ```python        
    # Steward Agent    
    sender_verkey, authdecrypted_faber_did_info_json = \
        await crypto.auth_decrypt(steward_handle, steward_faber_key, authcrypted_faber_did_info_json)
    faber_did_info = json.loads(authdecrypted_faber_did_info_json)
    ```

6. **Steward** asks the ledger for the Verification key of **Faber's** DID by calling ``did.key_for_did``.
    ```python        
    # Steward Agent    
    faber_verkey = await did.key_for_did(pool_handle, from_wallet, faber_did_info['did'])
    ```

7. **Steward** authenticates **Faber** by comparison of the Message Sender Verkey and the **Faber** Verkey received from the Ledger.
    ```python        
    # Steward Agent    
    assert sender_verkey == faber_verkey
    ```

8. **Steward** sends the corresponded NYM transaction to the Ledger with `TRUST ANCHOR` role.
Please note that despite the fact that the Steward is the sender of this transaction the owner of DID will be Faber as it uses Verkey provided by Faber.
    ```python    
    # Steward Agen
    nym_request = await ledger.build_nym_request(steward_did, decrypted_faber_did_info_json['did'],
                                                 decrypted_faber_did_info_json['verkey'], None, 'TRUST_ANCHOR')
    await ledger.sign_and_submit_request(pool_handle, steward_wallet, steward_did, nym_request)
    ```
At this point **Faber** has a DID related to his identity in the Ledger.

**Acme**, **Thrift Bank**, and **Government** must pass the same Onboarding process connection establishment with **Steward**.


[previous](04-onboarding-agents.ipynb) | [next](06-cred-schema-setup.ipynb)